# Использование последовательного анализа

В этом ноутбуке приведена простая _реализация последовательного анализа_,
поволяющая _сокращать время проведения_ тестов конверсий
_на десятки процентов_ (а то и _в разы_).
Здесь вы **можете попробовать** последовательный анализ,
сравнив длительность теста с классическим дизайном.

Код является частью обширного исследования последовательного анализа,
доступного на Github - [Xapulc/wald-sequential-probability-ratio-test/](https://github.com/Xapulc/wald-sequential-probability-ratio-test/). Также средний размер выборки при последовательном анализе можно оценить с помощью калькулятора размера выборки [ABnTester](https://abntester.com).

# Случай двух выборок

## Реализация последовательного анализа

In [ ]:
import numpy as np


class BinaryTwoSampleSprt(object):
    def __init__(self, p0, d, alpha=0.05, beta=0.2, alternative="two-sided",
                 initial_first_success_cnt=0, initial_first_sample_size=0,
                 initial_second_success_cnt=0, initial_second_sample_size=0,
                 initial_one_sample_success_cnt=0,
                 initial_one_sample_sample_size=0):
        """
        Последовательный анализ в случае двухвыборочной задачи

        :param p0: значение вероятности при гипотезе
        :param d: абсолютное значение MDE
        :param alpha: ограничение на вероятность ошибки I рода (уровень значимости)
        :param beta: ограничение на вероятность ошибки II рода (1 - мощность)
        :param alternative: наименование односторонней альтернативы
                            less: правосторонняя альтернатива p1 < p2
                            greater: левосторонняя альтернатива p1 > p2
                            two-sided: двусторонняя альтернатива p1 != p2
        :param initial_first_success_cnt: изначальное количество "успехов" в первой выборке
        :param initial_first_sample_size: изначальный размер первой выборки
        :param initial_second_success_cnt: изначальное количество "успехов" во второй выборке
        :param initial_second_sample_size: изначальный размер второй выборки
        """
        # Параметры последовательного теста
        self.p0 = p0
        self.d = np.abs(d)
        self.alpha = alpha
        self.beta = beta
        self.alternative = alternative

        # Параметры текущего состояния теста
        self.first_success_cnt = initial_first_success_cnt
        self.first_sample_size = initial_first_sample_size

        self.second_success_cnt = initial_second_success_cnt
        self.second_sample_size = initial_second_sample_size

        self.one_sample_success_cnt = initial_one_sample_success_cnt
        self.one_sample_sample_size = initial_one_sample_sample_size

        self.first_sample_buf = []
        self.second_sample_buf = []

        # Принятие решения
        self.stop_first_success_cnt = self.first_success_cnt
        self.stop_first_sample_size = self.first_sample_size

        self.stop_second_success_cnt = self.second_success_cnt
        self.stop_second_sample_size = self.second_sample_size

        self.decision_desc = "Тест продолжается"

        # Признак остановки последовательного анализа
        # для двусторонней альтернативы
        self.greater_stop_flg = False
        self.less_stop_flg = False

    def transform_two_sample_one_sided_mde(self, p_low, p_high):
        """
        Функция, вычисляющая MDE для одновыборочной задачи
        из параметров двухвыборочного последовательного анализа

        Вальд А.
        Последовательный анализ.
        – 1960. – С. 143-146.

        :param p0: значение вероятности при гипотезе
        :param d: абсолютное значение MDE
        :param alternative: наименование односторонней альтернативы
        :return: MDE одновыборочной задачи
        """
        p0_transformed = 1 / 2
        p_transformed = (1 - p_low) * p_high / ((1 - p_low) * p_high + p_low * (1 - p_high))
        d_transformed = np.abs(p_transformed - p0_transformed)

        return d_transformed

    def calc_one_sided_probs(self, alternative):
        """
        Функция для расчёта базовых значений вероятностей (конверсий)
        (нижней и верхней)

        :param alternative: наименование односторонней альтернативы
                            greater: правосторонняя альтернатива p > p0
                            less: левосторонняя альтернатива p < p0
        :return: нижнее значение вероятности, верхнее значение вероятности
        """

        if alternative == "greater":
            d_transformed = self.transform_two_sample_one_sided_mde(self.p0, self.p0+self.d)
            p_low = 1 / 2
            p_high = p_low + d_transformed
        elif alternative == "less":
            d_transformed = self.transform_two_sample_one_sided_mde(self.p0-self.d, self.p0)
            p_high = 1 / 2
            p_low = p_high - d_transformed
        else:
            raise ValueError(f"Неправильная альтернатива: {alternative}")

        return p_low, p_high

    def calc_one_sided_bounds(self, alpha, beta, alternative):
        """
        Функция для односторонней альтернативы
        рассчитывает пороговые значения,
        при пересечении которых тест останавливается и принимается решение

        :param alpha: ограничение на вероятность ошибки I рода (уровень значимости)
        :param beta: ограничение на вероятность ошибки II рода (1 - мощность)
        :param alternative: наименование односторонней альтернативы
                            greater: правосторонняя альтернатива p > p0
                            less: левосторонняя альтернатива p < p0
        :return: нижняя граница, верхняя граница
        """

        if alternative == "greater":
            low_bound = np.log(beta / (1 - alpha))
            high_bound = np.log((1 - beta) / alpha)
        elif alternative == "less":
            low_bound = np.log(alpha / (1 - beta))
            high_bound = np.log((1 - alpha) / beta)
        else:
            raise ValueError(f"Неправильная альтернатива: {alternative}")

        return low_bound, high_bound

    def calc_one_sided_curve(self, success_cnt, sample_size, alternative):
        """
        Функция для расчёта значений логарифмического отношения правдоподобий

        :param success_cnt: количество "успехов"
        :param sample_size: размер выборки
        :param alternative: наименование односторонней альтернативы
                            greater: правосторонняя альтернатива p > p0
                            less: левосторонняя альтернатива p < p0
        :return:
        """
        p_low, p_high = self.calc_one_sided_probs(alternative)

        # Логарифмическое отношение правдоподобий для бернуллиевских величин
        curve = success_cnt * np.log(p_high / p_low) \
                + (sample_size - success_cnt) * np.log((1 - p_high) / (1 - p_low))

        return curve

    def append(self, x, first_sample_flg):
        """
        Добавление нового элемента выборки
        с принятием решения о возможности
        остановки последовательного теста

        :param x: значение нового элемента выборки
        :param first_sample_flg: флаг того, что x из первой выборки
        :return: описание принятого решения
        """

        # Обновление общей статистики теста
        if first_sample_flg:
            self.first_success_cnt += x
            self.first_sample_size += 1
        else:
            self.second_success_cnt += x
            self.second_sample_size += 1

        # Если тест продолжается, обновляем расчёты
        if self.decision_desc == "Тест продолжается":
            # Так как тест ещё не остановлен,
            # обновляем статистику теста до принятого решения
            self.stop_first_success_cnt = self.first_success_cnt
            self.stop_first_sample_size = self.first_sample_size

            self.stop_second_success_cnt = self.second_success_cnt
            self.stop_second_sample_size = self.second_sample_size

            if (first_sample_flg and len(self.second_sample_buf) == 0) \
                or (not first_sample_flg and len(self.first_sample_buf) == 0):
                if first_sample_flg and len(self.second_sample_buf) == 0:
                    self.first_sample_buf.append(x)
                else:
                    self.second_sample_buf.append(x)
            else:
                if first_sample_flg and len(self.second_sample_buf) > 0:
                    first_value = x
                    second_value = self.second_sample_buf.pop(0)
                else:
                    first_value = self.first_sample_buf.pop(0)
                    second_value = x

                # Переход к одновыборочной задаче
                self.one_sample_success_cnt += first_value * (1 - second_value)
                self.one_sample_sample_size += 1 if first_value != second_value else 0

                if self.alternative != "two-sided":
                    # Если альтернатива одностороняя,
                    # то решение принимается по одному расчёту
                    # логарифмического отношения правдоподобий
                    curve = self.calc_one_sided_curve(self.one_sample_success_cnt,
                                                      self.one_sample_sample_size,
                                                      self.alternative)
                    low_bound, high_bound = self.calc_one_sided_bounds(self.alpha,
                                                                       self.beta,
                                                                       self.alternative)

                    # Если значение логарифмического отношения правдоподобий
                    # пересекает одну из границ,
                    # тест останавливается с принятием решения
                    if curve > high_bound:
                        if self.alternative == "greater":
                            self.decision_desc = "Тест остановлен, справедлива альтернатива p1 > p2"
                        else:
                            self.decision_desc = "Тест остановлен, справедлива гипотеза p1 >= p2"
                    elif curve < low_bound:
                        if self.alternative == "greater":
                            self.decision_desc = "Тест остановлен, справедлива гипотеза p1 <= p2"
                        else:
                            self.decision_desc = "Тест остановлен, справедлива альтернатива p1 < p2"
                else:
                    # Если альтернатива двусторонняя,
                    # то мы параллельно "проводим" два последовательных анализа:
                    # p0 против p0+d (alternative = "greater"),
                    # p0-d против p0 (alternative = "less")
                    greater_curve = self.calc_one_sided_curve(self.one_sample_success_cnt,
                                                              self.one_sample_sample_size,
                                                              alternative="greater")
                    greater_low_bound, greater_high_bound = self.calc_one_sided_bounds(self.alpha/2,
                                                                                       self.beta,
                                                                                       alternative="greater")

                    less_curve = self.calc_one_sided_curve(self.one_sample_success_cnt,
                                                           self.one_sample_sample_size,
                                                           alternative="less")
                    less_low_bound, less_high_bound = self.calc_one_sided_bounds(self.alpha/2,
                                                                                 self.beta,
                                                                                 alternative="less")

                    # Если тест для alternative = "greater" ранее не завершён,
                    # а сейчас произошло пересечение верхней границы,
                    # то останавливаем тест с решением о стат. значимом росте
                    if not self.greater_stop_flg and greater_curve > greater_high_bound:
                        self.decision_desc = "Тест остановлен, справедлива альтернатива p1 > p2"

                    # Если тест для alternative = "less" ранее не завершён,
                    # а сейчас произошло пересечение нижней границы,
                    # то останавливаем тест с решением о стат. значимом падении
                    if not self.less_stop_flg and less_curve < less_low_bound:
                        self.decision_desc = "Тест остановлен, справедлива альтернатива p1 < p2"

                    # Если для какой-то из альтернатив тест был ранее завершён,
                    # но тест с двусторонней альтернативой продолжается,
                    # то ранее было пересечение границы, соответствующее p1 = p2
                    # Поэтому если для какой-то альтернативы тест завершился ранее,
                    # а сейчас для другой альтернативы
                    # есть пересечение границы, соответствующее p1 = p2,
                    # то мы можем завершить тест с принятием решения p1 = p2
                    if self.greater_stop_flg and less_curve > less_high_bound:
                        self.decision_desc = "Тест остановлен, справедлива гипотеза p1 = p2"
                    if self.less_stop_flg and greater_curve < greater_low_bound:
                        self.decision_desc = "Тест остановлен, справедлива гипотеза p1 = p2"

                    # Если ни для какой альтернативы тест ранее не был завершён,
                    # а сейчас для обеих альтернатив есть пересечение границы при p1 = p2,
                    # то мы можем завершить тест с принятием решения p1 = p2
                    if not self.greater_stop_flg and greater_curve < greater_low_bound \
                        and self.less_stop_flg and less_curve > less_high_bound:
                        self.decision_desc = "Тест остановлен, справедлива гипотеза p1 = p2"

                    # Завершаем тест для тех альтернатив,
                    # для которых есть пересечение хотя бы одной из границ
                    if greater_curve > greater_high_bound or greater_curve < greater_low_bound:
                        self.greater_stop_flg = True
                    if less_curve > less_high_bound or less_curve < less_low_bound:
                        self.less_stop_flg = True

        return self.decision_desc

    def append_list(self, x_list, y_list):
        """
        Добавление списка из новых элементов для обеих вариаций
        с принятием решения о возможности
        остановки последовательного теста

        :param x_list: список значений новых элементов первой выборки
        :param y_list: список значений новых элементов второй выборки
        :return: описание принятого решения
        """

        decision_desc = self.decision_desc
        for x in x_list:
            decision_desc = self.append(x, first_sample_flg=True)
        for y in y_list:
            decision_desc = self.append(y, first_sample_flg=False)

        return decision_desc


## Применение

In [ ]:
from scipy.stats import bernoulli


p0 = 0.07                      # Базовая вероятность (историческая конверсия)
d = 0.005                      # Абсолютное значение MDE
alpha = 0.05                   # Уровень значимости (ограчниение на вероятность ошибки I рода)
beta = 0.2                     # 1 - мощность (ограчниение на вероятность ошибки II рода)
alternative = "two-sided"      # Наименование альтернативы
# Эти параметры можно заменить на свои

# Класс последовательного анализа
sprt = BinaryTwoSampleSprt(p0, d, alpha, beta,
                           alternative=alternative)

decision_desc = "Тест продолжается" # Описание текущего результата теста
p1 = p0                             # Истинное значение конверсии на первой выборке
p2 = p0 - d                         # Истинное значение конверсии на второй выборке
# Истинные конверсии тоже можно заменить на свои

while decision_desc == "Тест продолжается":  # Продолжаем тест пока не принято решение об остановке
    sample_num = 1 + bernoulli.rvs(0.5)      # Разыгрываем то, в какую вариацию попадёт новый участник теста
    first_sample_flg = sample_num == 1       # Флаг попадания в первую выборку

    # Розыгрыш значения целевой метрики теста
    # в зависимости от номера выборки
    if first_sample_flg:
        x = bernoulli.rvs(p1)
    else:
        x = bernoulli.rvs(p2)

    # Добавление значения целевой метрики теста
    # в последовательный анализ
    decision_desc = sprt.append(x, first_sample_flg)
    # Можно добавлять сразу список значений:
    # sprt.append_list(x_list, y_list) -> принятое решение по всем данным

    if decision_desc != "Тест продолжается":
        stop_flg = True

print(decision_desc)
print(f"Длительность теста для 1 выборки: {sprt.stop_first_sample_size}, "
      + f"для 2 выборки: {sprt.stop_second_sample_size}.")

Тест остановлен, справедлива альтернатива p1 > p2
Длительность теста для 1 выборки: 35386, для 2 выборки: 35136.


1. Сравните _принятое решение_ по итогу теста
с _истинными значениями_ конверсий `p1` и `p2`. **Правильное ли решение** было принято?
2. Сравните _длительность последовательного_ теста
с _длительностью_ теста по _классической_ методологии. **Есть ли ускорение**?

Длительность теста по классической методологии
можно рассчитать в [онлайн-калькуляторе](https://abntester.com/#/calculation/two-sample?p=7&mde=0.5&alpha=5&beta=20&leftProportion=50&type=BINARY&alternative=TWO_SIDED&showResult=1).

P.S. Длительность теста при последовательном анализе случайна,
и может превышать значение из калькулятора.
Однако же, это происходит редко.

# Случай одной выборки

## Реализация последовательного анализа

In [ ]:
import numpy as np


class BinaryOneSampleSprt(object):
    def __init__(self, p0, d, alpha=0.05, beta=0.2, alternative="two-sided",
                 initial_success_cnt=0, initial_sample_size=0):
        """
        Последовательный анализ в случае одновыборочной задачи

        :param p0: значение вероятности при гипотезе
        :param d: абсолютное значение MDE
        :param alpha: ограничение на вероятность ошибки I рода (уровень значимости)
        :param beta: ограничение на вероятность ошибки II рода (1 - мощность)
        :param alternative: наименование односторонней альтернативы
                            greater: правосторонняя альтернатива p > p0
                            less: левосторонняя альтернатива p < p0
                            two-sided: двусторонняя альтернатива p != p0
        :param initial_success_cnt: изначальное количество "успехов"
        :param initial_sample_size: изначальный размер выборки
        """
        # Параметры последовательного теста
        self.p0 = p0
        self.d = np.abs(d)
        self.alpha = alpha
        self.beta = beta
        self.alternative = alternative

        # Параметры текущего состояния теста
        self.success_cnt = initial_success_cnt
        self.sample_size = initial_sample_size

        # Принятие решения
        self.stop_success_cnt = self.success_cnt
        self.stop_sample_size = self.sample_size
        self.decision_desc = "Тест продолжается"

        # Признак остановки последовательного анализа
        # для двусторонней альтернативы
        self.greater_stop_flg = False
        self.less_stop_flg = False

    def calc_one_sided_probs(self, alternative):
        """
        Функция для расчёта базовых значений вероятностей (конверсий)
        (нижней и верхней)

        :param alternative: наименование односторонней альтернативы
                            greater: правосторонняя альтернатива p > p0
                            less: левосторонняя альтернатива p < p0
        :return: нижнее значение вероятности, верхнее значение вероятности
        """

        if alternative == "greater":
            p_low = self.p0
            p_high = self.p0 + self.d
        elif alternative == "less":
            p_low = self.p0 - self.d
            p_high = self.p0
        else:
            raise ValueError(f"Неправильная альтернатива: {alternative}")

        return p_low, p_high

    def calc_one_sided_bounds(self, alpha, beta, alternative):
        """
        Функция для односторонней альтернативы
        рассчитывает пороговые значения,
        при пересечении которых тест останавливается и принимается решение

        :param alpha: ограничение на вероятность ошибки I рода (уровень значимости)
        :param beta: ограничение на вероятность ошибки II рода (1 - мощность)
        :param alternative: наименование односторонней альтернативы
                            greater: правосторонняя альтернатива p > p0
                            less: левосторонняя альтернатива p < p0
        :return: нижняя граница, верхняя граница
        """

        if alternative == "greater":
            low_bound = np.log(beta / (1 - alpha))
            high_bound = np.log((1 - beta) / alpha)
        elif alternative == "less":
            low_bound = np.log(alpha / (1 - beta))
            high_bound = np.log((1 - alpha) / beta)
        else:
            raise ValueError(f"Неправильная альтернатива: {alternative}")

        return low_bound, high_bound

    def calc_one_sided_curve(self, success_cnt, sample_size, alternative):
        """
        Функция для расчёта значений логарифмического отношения правдоподобий

        :param success_cnt: количество "успехов"
        :param sample_size: размер выборки
        :param alternative: наименование односторонней альтернативы
                            greater: правосторонняя альтернатива p > p0
                            less: левосторонняя альтернатива p < p0
        :return:
        """
        p_low, p_high = self.calc_one_sided_probs(alternative)

        # Логарифмическое отношение правдоподобий для бернуллиевских величин
        curve = success_cnt * np.log(p_high / p_low) \
                + (sample_size - success_cnt) * np.log((1 - p_high) / (1 - p_low))

        return curve

    def append(self, x):
        """
        Добавление нового элемента выборки
        с принятием решения о возможности
        остановки последовательного теста

        :param x: значение нового элемента выборки
        :return: описание принятого решения
        """

        # Обновление общей статистики теста
        self.success_cnt += x
        self.sample_size += 1

        # Если тест продолжается, обновляем расчёты
        if self.decision_desc == "Тест продолжается":
            # Так как тест ещё не остановлен,
            # обновляем статистику теста до принятого решения
            self.stop_success_cnt = self.success_cnt
            self.stop_sample_size = self.sample_size

            if self.alternative != "two-sided":
                # Если альтернатива одностороняя,
                # то решение принимается по одному расчёту
                # логарифмического отношения правдоподобий
                curve = self.calc_one_sided_curve(self.success_cnt, self.sample_size, self.alternative)
                low_bound, high_bound = self.calc_one_sided_bounds(self.alpha, self.beta, self.alternative)

                # Если значение логарифмического отношения правдоподобий
                # пересекает одну из границ,
                # тест останавливается с принятием решения
                if curve > high_bound:
                    if self.alternative == "greater":
                        self.decision_desc = "Тест остановлен, справедлива альтернатива p > p0"
                    else:
                        self.decision_desc = "Тест остановлен, справедлива гипотеза p >= p0"
                elif curve < low_bound:
                    if self.alternative == "greater":
                        self.decision_desc = "Тест остановлен, справедлива гипотеза p <= p0"
                    else:
                        self.decision_desc = "Тест остановлен, справедлива альтернатива p < p0"
            else:
                # Если альтернатива двусторонняя,
                # то мы параллельно "проводим" два последовательных анализа:
                # p0 против p0+d (alternative = "greater"),
                # p0-d против p0 (alternative = "less")
                greater_curve = self.calc_one_sided_curve(self.success_cnt, self.sample_size, alternative="greater")
                greater_low_bound, greater_high_bound = self.calc_one_sided_bounds(self.alpha/2, self.beta, alternative="greater")

                less_curve = self.calc_one_sided_curve(self.success_cnt, self.sample_size, alternative="less")
                less_low_bound, less_high_bound = self.calc_one_sided_bounds(self.alpha/2, self.beta, alternative="less")

                # Если тест для alternative = "greater" ранее не завершён,
                # а сейчас произошло пересечение верхней границы,
                # то останавливаем тест с решением о стат. значимом росте
                if not self.greater_stop_flg and greater_curve > greater_high_bound:
                    self.decision_desc = "Тест остановлен, справедлива альтернатива p > p0"

                # Если тест для alternative = "less" ранее не завершён,
                # а сейчас произошло пересечение нижней границы,
                # то останавливаем тест с решением о стат. значимом падении
                if not self.less_stop_flg and less_curve < less_low_bound:
                    self.decision_desc = "Тест остановлен, справедлива альтернатива p < p0"

                # Если для какой-то из альтернатив тест был ранее завершён,
                # но тест с двусторонней альтернативой продолжается,
                # то ранее было пересечение границы, соответствующее p = p0
                # Поэтому если для какой-то альтернативы тест завершился ранее,
                # а сейчас для другой альтернативы
                # есть пересечение границы, соответствующее p = p0,
                # то мы можем завершить тест с принятием решения p = p0
                if self.greater_stop_flg and less_curve > less_high_bound:
                    self.decision_desc = "Тест остановлен, справедлива гипотеза p = p0"
                if self.less_stop_flg and greater_curve < greater_low_bound:
                    self.decision_desc = "Тест остановлен, справедлива гипотеза p = p0"

                # Если ни для какой альтернативы тест ранее не был завершён,
                # а сейчас для обеих альтернатив есть пересечение границы при p = p0,
                # то мы можем завершить тест с принятием решения p = p0
                if not self.greater_stop_flg and greater_curve < greater_low_bound \
                    and self.less_stop_flg and less_curve > less_high_bound:
                    self.decision_desc = "Тест остановлен, справедлива гипотеза p = p0"

                # Завершаем тест для тех альтернатив,
                # для которых есть пересечение хотя бы одной из границ
                if greater_curve > greater_high_bound or greater_curve < greater_low_bound:
                    self.greater_stop_flg = True
                if less_curve > less_high_bound or less_curve < less_low_bound:
                    self.less_stop_flg = True

        return self.decision_desc

    def append_list(self, x_list):
        """
        Добавление списка из новых элементов выборки
        с принятием решения о возможности
        остановки последовательного теста

        :param x_list: список значений новых элементов выборки
        :return: описание принятого решения
        """

        decision_desc = self.decision_desc
        for x in x_list:
            decision_desc = self.append(x)

        return decision_desc


## Применение

In [ ]:
from scipy.stats import bernoulli


p0 = 0.07                      # Базовая вероятность (историческая конверсия)
d = 0.005                      # Абсолютное значение MDE
alpha = 0.05                   # Уровень значимости (ограчниение на вероятность ошибки I рода)
beta = 0.2                     # 1 - мощность (ограчниение на вероятность ошибки II рода)
alternative = "less"           # Наименование альтернативы
# Эти параметры можно заменить на свои

# Класс последовательного анализа
sprt = BinaryOneSampleSprt(p0, d, alpha, beta,
                           alternative=alternative)

decision_desc = "Тест продолжается" # Описание текущего результата теста
p = p0                              # Истинное значение конверсии
# Значение p можно заменить на своё

while decision_desc == "Тест продолжается":
    # Генерация значения целевой метрики
    # с истинной конверсией p
    x = bernoulli.rvs(p)

    # Добавление значения целевой метрики теста
    # в последовательный анализ
    decision_desc = sprt.append(x)
    # Можно добавлять сразу список значений:
    # sprt.append_list(x_list) -> принятое решение по всем данным

    if decision_desc != "Тест продолжается":
        stop_flg = True

print(decision_desc)
print(f"Длительность теста: {sprt.stop_sample_size}")

Тест остановлен, справедлива гипотеза p >= p0
Длительность теста: 3958


1. Сравните _принятое решение_ по итогу теста
с _истинным значением_ конверсии `p`. **Правильное ли решение** было принято?
2. Сравните _длительность последовательного_ теста
с _длительностью_ теста по _классической_ методологии. **Есть ли ускорение**?

Длительность теста по классической методологии
можно рассчитать в [онлайн-калькуляторе](https://abntester.com/#/calculation/one-sample?p=7&mde=0.5&alpha=5&beta=20&type=BINARY&alternative=LEFT_SIDED&showResult=1).

P.S. Длительность теста при последовательном анализе случайна,
и может превышать значение из калькулятора.
Однако же, это происходит редко.